# `*`模块加载特殊内容

从加载机制上看其实python解释器可以加载任何内容,只要定义好相应的finder和loader即可.这种扩展方式叫做`import hook`,官方其实也提供了几个`import hook`来实现一些特殊需求.本文将介绍这个

## 模块加载数据

就和C/C++中一些常量数据甚至文件内容直接写在头文件中可以加快运行时的速度一样,在python的导入时直接将需要的数据导入其实也是一种提高运行时效率的方法.

python中有标准库`pkgutil.get_data(package,path)`,可以帮我们实现这个需求.

我们有一个html文件叫"base.html"这边我们使用`pkgutil`来加载它

1. 将其放入一个模块内
    我们将其放入一个叫data_demo的模块中
    ```shell
    data_demo-|
              |-__init__.py
              |-base.html
    ```
    
2. 使用pkgutil.get_data加载数据

In [5]:
import pkgutil

template = pkgutil.get_data('data_demo', 'base.html')
print(template.decode('utf-8'))

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML//EN">
<html> <head>
<title>PyMOTW Template</title>
</head>

<body>
<h1>Example Template</h1>

<p>This is a sample data file.</p>

</body>
</html>


### 加载zip包中的数据模块

python默认就可以加载zip包中的模块.这也就为大文件的导入提供了方便.


我们将`data_demo`文件中的文件打包到zip归档中

```shell
zipdata_demo.zip--|
                  |-data_demo-|
                              |-__init__.py
                              |-base.html
```
然后还是使用`pkgutil`来读取数据,注意需要先将`zipdata_demo.zip`加入模块可以访问的位置

In [6]:
import pkgutil
import sys
sys.path.insert(0, 'zipdata_demo.zip')

In [7]:
import data_demo

In [8]:
emplate = pkgutil.get_data('data_demo', 'base.html')
print(emplate.decode('utf-8'))

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML//EN">
<html> <head>
<title>PyMOTW Template</title>
</head>

<body>
<h1>Example Template</h1>

<p>This is a sample data file.</p>

</body>
</html>


## 模块加载zip包中的python模块

上面的例子我们可以看到python自带加载zip中模块的能力,但每次加载都要显式的写上代码这样相当的不优雅

```python
import sys
sys.path.insert(0, 'xxxx.zip')
```

python提供了标准库`zipimport`来解决这个问题

In [1]:
import zipimport

importer = zipimport.zipimporter('zipdata_demo.zip')
module = importer.load_module('data_demo')

In [2]:
module.__path__

['zipdata_demo.zip/data_demo']

In [4]:
import pkgutil
emplate = pkgutil.get_data('data_demo', 'base.html')
print(emplate.decode('utf-8'))

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML//EN">
<html> <head>
<title>PyMOTW Template</title>
</head>

<body>
<h1>Example Template</h1>

<p>This is a sample data file.</p>

</body>
</html>
